In [1]:
import pandas as pd
import json
import math
import sqlite3
import datetime
year = 2025
pd.set_option('display.max_columns', None)
cnx = sqlite3.connect('results.db')

## Janowiec Wielkopolski

In [2]:
results = []
with open('janowiec-wielkopolski.txt') as f:
    for line in f.readlines():
        parts = line.split('\t')
        results.append([parts[5] + ' ' + parts[7], parts[9], (parts[15] or parts[14])[:1], (parts[19] or parts[18])[:3], parts[11] or None, int(next(p for p in parts if len(p.strip()) == 4 and p[:2] in {'19', '20'})), parts[-1].strip() or parts[-2].strip()])

janowiec = pd.DataFrame(results, columns=[
    'name',
    'city',
    'gender',
    'category',
    'team',
    'year_of_birth',
    'result',
])
assert not any(janowiec['name'].duplicated())
# sorted(janowiec[janowiec['team'].notnull()]['team'].unique())
janowiec.loc[janowiec['team'] == 'TRACK GANG', 'team'] = 'TRACK GANG TORUŃ'
janowiec.loc[janowiec['team'] == 'WATAHA ŁĄCZY NAS BIEGANIE', 'team'] = 'WATAHA - ŁĄCZY NAS BIEGANIE'
janowiec.loc[janowiec['team'] == 'EKO- OPAŁ PŁONNE', 'team'] = 'EKO-OPAŁ PŁONNE'
janowiec.loc[janowiec['name'] == 'KRIGER- PRZYBYLSKA ANNA', 'name'] = 'KRIGER-PRZYBYLSKA ANNA'

janowiec

,name,city,gender,category,team,year_of_birth,result
0,ABRAMOWSKI DAMIAN,ŻNIN,M,M30,TT SODA,1991,00:37:45
1,ADAMSKA IZABELA,JANOWIEC WIELKOPOLSKI,K,K40,None,1985,00:47:33
2,ADAMSKA ANNA,KŁECKO,K,K40,LECHITA BIEGA KŁECKO,1985,00:57:09
3,ADAMSKI ADAM,ŻNIN,M,M30,None,1986,00:47:48
4,ANGIEL WŁADYSŁAW,BYDGOSZCZ,M,M80,None,1937,01:40:32
...,...,...,...,...,...,...,...
267,ZMUDZIŃSKA JOANNA,PIECHCIN,K,K40,None,1983,00:45:37
268,ŚWIĘCICKI ANDRZEJ,BYDGOSZCZ,M,M50,BYDGOSKIE ŻÓŁWIE,1973,00:50:43
269,ŁASIŃSKI DARIUSZ,KOWALEWO,M,M30,None,1991,00:44:20
270,ŻUCHOWSKI JÓZEF,PŁONNE,M,M50,EKO-OPAŁ PŁONNE,1971,00:41:23


## Fabianki

In [3]:
fabianki = pd.read_csv('fabianki.txt', sep='\t', names=['c0', 'c1', 'name', 'gender', 'c2', 'c3', 'category', 'c5', 'team', 'c6', 'c7', 'c8', 'c9', 'result', 'c11'])
fabianki = fabianki[['name', 'gender', 'category', 'team', 'result']]
fabianki['name'] = fabianki['name'].str.upper()
fabianki['team'] = fabianki['team'].str.upper()
fabianki = fabianki[fabianki['result'].notnull()]
fabianki['category'] = fabianki['category'].str.slice(0, 3)

fabianki.loc[fabianki['team'] == 'PAŁUKI RUNNIG TEAM', 'team'] = 'PAŁUKI RUNNING TEAM'
fabianki.loc[fabianki['team'] == 'T G SOKÓŁ GNIEWKOWO', 'team'] = 'TG SOKÓŁ GNIEWKOWO'
fabianki.loc[fabianki['team'] == 'T.G.SOKÓŁ GNIEWKOWO', 'team'] = 'TG SOKÓŁ GNIEWKOWO'
fabianki.loc[fabianki['team'] == 'WKB MARATONCZYK WLOCLAWEK', 'team'] = 'WKB MARATOŃCZYK WŁOCŁAWEK'
fabianki.loc[fabianki['team'] == 'WKB MARATONCZYK WŁOCŁAWEK', 'team'] = 'WKB MARATOŃCZYK WŁOCŁAWEK'
fabianki.loc[fabianki['name'] == 'BLASZCZAK STANISLAW', 'name'] = 'BŁASZCZAK STANISŁAW'
fabianki.loc[fabianki['name'] == 'WŁADYSŁAW ANGIEL', 'name'] = 'ANGIEL WŁADYSŁAW'

any(fabianki['name'].duplicated())

fabianki

,name,gender,category,team,result
0,KLEDZIK ZBIGNIEW,M,M30,PAŁUKI RUNNING TEAM,00:32:21
1,TOMASZEWSKI MIŁOSZ,M,M30,GKS FABIANKI,00:33:05
2,TOMASZEWSKI MACIEJ,M,M30,TASMAN TEAM,00:33:46
3,KUNECKI DARIUSZ,M,M40,KUNA TEAM WŁOCŁAWEK,00:34:42
4,SZTOLCMAN PIOTR,M,M30,GKS FABIANKI,00:34:55
...,...,...,...,...,...
161,GLAZIK RENATA,K,K50,FABIANKI,01:18:26
162,WITKOWSKA EDYTA,K,K50,FABIANKI,01:18:32
163,WITKOWSKA JULIA,K,K16,FABIANKI,01:18:33
164,ANDELBRAT BEATA,K,K50,GRUDZIADZ,01:18:33


## Radziejów

In [4]:
results = []
with open('radziejow.json') as f:
    data = json.load(f)
    for row in data['data']:
        # for idx, f in enumerate(row):
        #     print(idx, f)
        # xxx
        if len(row) != 19:
            continue
        if row[14] == 'DNS':
            continue
        results.append([
            row[3].upper(),
            row[8].upper(),
            row[11][:1],
            row[7].upper() if row[7] else None,
            row[4],
            row[14].zfill(8),
        ])

radziejow = pd.DataFrame(results, columns=[
    'name',
    'city',
    'gender',
    'team',
    'year_of_birth',
    'result_radziejow',
])

fixes = [
    ('team', 'SOLTYS', 'SOŁTYS SUCHARZEWO'),
    ('team', 'WATAHA ŁĄCZY NAS BIEGANIE', 'WATAHA - ŁĄCZY NAS BIEGANIE'),
    ('city', 'WŁOCŁAWK', 'WŁOCŁAWEK'),
    ('name', 'GOLEBIEWSKI MICHAL', 'GOŁĘBIEWSKI MICHAŁ'),
    ('name', 'HUSSEIN ABDALLAH', 'HUSSEIN ABDALLA'),
    ('name', 'DLONIAK EDMUND', 'DŁONIAK EDMUND'),
    ('name', 'ZIÓŁKOWSKI MIECZYSLAW', 'ZIÓŁKOWSKI MIECZYSŁAW'),
]
for field, from_, to_ in fixes:
    radziejow.loc[radziejow[field] == from_, field] = to_

radziejow

,name,city,gender,team,year_of_birth,result_radziejow
0,RATAJCZYK PRZEMYSŁAW,BYDGOSZCZ,M,TKKF KOLEJARZ BYDGOSZCZ,1993,00:32:48
1,KLEDZIK ZBIGNIEW,KIERZKOWO,M,PAŁUKI RUNNING TEAM,1991,00:33:00
2,TOMASZEWSKI MACIEJ,TORUŃ,M,TASMAN TEAM,1988,00:33:04
3,JUREK ŁUKASZ,JANIKOWO,M,STOWARZYSZENIE SIŁA,1989,00:34:00
4,STANISZEWSKI SŁAWOMIR,ŻNIN,M,PAŁUKI RUNNING TEAM,1989,00:35:23
...,...,...,...,...,...,...
159,BONIECKA TERESA,RYPIN,K,RKS LECH RYPIN,1974,01:11:14
160,KRZYŻANIAK BOGDAN,ZAKRZEWO,M,None,1961,01:11:56
161,BŁASZCZAK STANISŁAW,SKEPE,M,WKB MARATOŃCZYK WŁOCŁAWEK,1946,01:12:52
162,LENC JANUSZ,BYDGOSZCZ,M,TKKF KOLEJARZ BYDGOSZCZ,1953,01:23:52


## Merge

In [5]:
after_2 = janowiec.merge(fabianki, on=['name', 'gender'], how='outer', suffixes=('_janowiec', '_fabianki'))

In [6]:
after_2[after_2['team_janowiec'].notnull() & after_2['team_fabianki'].notnull() & (after_2['team_janowiec'] != after_2['team_fabianki'])]

,name,city,gender,category_janowiec,team_janowiec,year_of_birth,result_janowiec,category_fabianki,team_fabianki,result_fabianki
72,GRABARSKA MAGDALENA,BRZOZÓWKA,K,K30,TRACK GANG TORUŃ,1986.0,00:47:21,K30,BRZOZÓWKA,00:45:24
118,KARPIŃSKI PAWEŁ,GOLUB-DOBRZYŃ,M,M20,ZAKOLATEAMFAMILIA,1996.0,00:39:14,M16,#ZAKOLATEAM,00:38:35
320,TOMASZEWSKI WOJCIECH,TORUŃ,M,M60,START TEAM WĄBRZEŹNO,1964.0,00:41:14,M60,TASMAN TEAM,00:39:40


In [7]:
after_2[after_2['category_janowiec'].notnull() & after_2['category_fabianki'].notnull() & (after_2['category_janowiec'] != after_2['category_fabianki'])]

,name,city,gender,category_janowiec,team_janowiec,year_of_birth,result_janowiec,category_fabianki,team_fabianki,result_fabianki
118,KARPIŃSKI PAWEŁ,GOLUB-DOBRZYŃ,M,M20,ZAKOLATEAMFAMILIA,1996.0,00:39:14,M16,#ZAKOLATEAM,00:38:35
234,PIETRZAK MICHAŁ,WŁOCŁAWEK,M,M20,KUNA TEAM WŁOCŁAWEK,2005.0,00:36:42,M16,KUNA TEAM WŁOCŁAWEK,00:36:28


In [8]:
after_2['team'] = after_2['team_fabianki'].combine_first(after_2['team_janowiec'])
after_2['category'] = after_2['category_fabianki'].combine_first(after_2['category_janowiec'])
after_2 = after_2.drop(labels=['category_janowiec', 'team_janowiec', 'category_fabianki', 'team_fabianki'], axis=1)
after_2['year_of_birth'] = after_2['year_of_birth'].astype('Int64')

#### Radziejów

In [9]:
after_3 = after_2.merge(radziejow, on=['name', 'gender'], how='outer', suffixes=('_m', '_radziejow'))
after_3['year_of_birth_radziejow'] = after_3['year_of_birth_radziejow'].astype('Int64')
after_3

,name,city_m,gender,year_of_birth_m,result_janowiec,result_fabianki,team_m,category,city_radziejow,team_radziejow,year_of_birth_radziejow,result_radziejow
0,ABRAMOWSKI DAMIAN,ŻNIN,M,1991,00:37:45,NaN,TT SODA,M30,NaN,NaN,<NA>,NaN
1,ADAMSKA ANNA,KŁECKO,K,1985,00:57:09,NaN,LECHITA BIEGA KŁECKO,K40,NaN,NaN,<NA>,NaN
2,ADAMSKA IZABELA,JANOWIEC WIELKOPOLSKI,K,1985,00:47:33,NaN,None,K40,NaN,NaN,<NA>,NaN
3,ADAMSKI ADAM,ŻNIN,M,1986,00:47:48,NaN,None,M30,NaN,NaN,<NA>,NaN
4,ANDELBRAT BEATA,NaN,K,<NA>,NaN,01:18:33,GRUDZIADZ,K50,NaN,NaN,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
444,ŚWIĘCICKI ANDRZEJ,BYDGOSZCZ,M,1973,00:50:43,NaN,BYDGOSKIE ŻÓŁWIE,M50,NaN,NaN,<NA>,NaN
445,ŻABIŃSKI RADOSŁAW,NaN,M,<NA>,NaN,NaN,NaN,NaN,WŁOCŁAWEK,WKB MARATOŃCZYK WŁOCŁAWEK,1986,00:51:29
446,ŻUCHOWSKI BARTOSZ,NaN,M,<NA>,NaN,00:37:34,PŁONNE,M16,PŁONNE,None,1996,00:38:14
447,ŻUCHOWSKI JÓZEF,PŁONNE,M,1971,00:41:23,00:40:08,EKO-OPAŁ PŁONNE,M50,NaN,NaN,<NA>,NaN


In [10]:
after_3[after_3['team_m'].notnull() & after_3['team_radziejow'].notnull() & (after_3['team_m'] != after_3['team_radziejow'])]

,name,city_m,gender,year_of_birth_m,result_janowiec,result_fabianki,team_m,category,city_radziejow,team_radziejow,year_of_birth_radziejow,result_radziejow
233,MECHLIŃSKI BARTŁOMIEJ,NaN,M,<NA>,NaN,00:37:00,TORUŃ,M40,TORUŃ,BANANOWY WOJOWNIK,1979,00:37:37


In [11]:
after_3[after_3['year_of_birth_radziejow'].notnull() & after_3['year_of_birth_m'].notnull() & (after_3['year_of_birth_radziejow'] != after_3['year_of_birth_m'])]

,name,city_m,gender,year_of_birth_m,result_janowiec,result_fabianki,team_m,category,city_radziejow,team_radziejow,year_of_birth_radziejow,result_radziejow


In [12]:
after_3[after_3['city_radziejow'].notnull() & after_3['city_m'].notnull() & (after_3['city_radziejow'] != after_3['city_m'])]

,name,city_m,gender,year_of_birth_m,result_janowiec,result_fabianki,team_m,category,city_radziejow,team_radziejow,year_of_birth_radziejow,result_radziejow
89,GRUDZIŃSKI MACIEJ,ŁÓDŹ,M,1986,00:37:13,NaN,WKB MARATOŃCZYK WŁOCŁAWEK,M30,WŁOCŁAWEK,WKB MARATOŃCZYK WŁOCŁAWEK,1986,00:36:16
104,HUSSEIN ABDALLA,ODOLION,M,1987,00:44:03,NaN,PARKRUN TORUŃ,M30,KONRADOWO,PARKRUN TORUŃ,1987,00:41:54
105,HUSSEIN ANNA,ODOLION,K,1981,00:51:12,NaN,PARKRUN TORUŃ,K40,KONRADOWO,PARKRUN TORUŃ,1981,00:49:13


In [13]:
after_3['team'] = after_3['team_radziejow'].combine_first(after_3['team_m'])
after_3['year_of_birth'] = after_3['year_of_birth_radziejow'].combine_first(after_3['year_of_birth_m'])
after_3['city'] = after_3['city_radziejow'].combine_first(after_3['city_m'])
after_3 = after_3.drop(labels=['team_radziejow', 'team_m', 'year_of_birth_radziejow', 'year_of_birth_m', 'city_radziejow', 'city_m'], axis=1)

In [14]:
after_3[after_3['team'] == 'PARKRUN TORUŃ']

,name,gender,result_janowiec,result_fabianki,category,result_radziejow,team,year_of_birth,city
7,BAJEŃSKI DARIUSZ,M,NaN,NaN,NaN,00:48:30,PARKRUN TORUŃ,1993,TORUŃ
11,BARWIŃSKI LESZEK,M,NaN,00:42:45,M50,NaN,PARKRUN TORUŃ,<NA>,NaN
49,DEPERAS ALEKSANDER,M,NaN,00:37:30,M40,00:38:46,PARKRUN TORUŃ,1984,TORUŃ
66,DĘBICKI MACIEJ,M,00:42:30,NaN,M30,00:42:38,PARKRUN TORUŃ,1993,TORUŃ
71,FRĄK EDYTA,K,00:42:55,00:41:41,K30,00:42:47,PARKRUN TORUŃ,1988,TORUŃ
81,GOŁĘBIEWSKI MICHAŁ,M,NaN,00:40:41,M40,00:42:28,PARKRUN TORUŃ,1983,TORUŃ
94,GUTTFELD IGOR,M,00:39:45,00:39:24,M50,NaN,PARKRUN TORUŃ,1967,MŁYNIEC PIERWSZY
96,GÓRSKI MICHAŁ,M,NaN,00:48:25,M30,00:52:30,PARKRUN TORUŃ,1991,TORUŃ
104,HUSSEIN ABDALLA,M,00:44:03,NaN,M30,00:41:54,PARKRUN TORUŃ,1987,KONRADOWO
105,HUSSEIN ANNA,K,00:51:12,NaN,K40,00:49:13,PARKRUN TORUŃ,1981,KONRADOWO


In [15]:
sorted(after_3[after_3['team'].notnull()]['team'].unique())

['#SLIMAKIBARCIN',
 '#WKTM',
 '#ZAKOLATEAM',
 'ALTOM GNIEZNO',
 'AMATEUR RUNNERS WŁOCŁAWEK',
 'BACHORZEWO',
 'BANALNY TRENER',
 'BANANOWY WOJOWNIK',
 'BANICI ULTRA ADVENTURE',
 'BBL INOWROCŁAW',
 'BENEDEK-TEAM',
 'BIEGOWI PRZYJACIELE',
 'BOBROWNIKI',
 'BRZOZÓWKA',
 'BYDGOSKIE ŻÓŁWIE',
 'BYDGOSZCZ',
 'BYDGOSZCZ NA START',
 'BĄDKOWO',
 'CHOCEŃ',
 'CYBORGI',
 'DECATHLON TEAM INOWROCŁAW',
 'DOBRZYŃ NAD WISŁĄ',
 'DREAM TEAM CHAWŁODNO',
 'EKO-OPAŁ PŁONNE',
 'ERKA RUN TEAM',
 'FABIANKI',
 'FUNDACJA SODA',
 'GDAŃSK',
 'GKS FABIANKI',
 'GLIWEK',
 'GRUDZIADZ',
 'JACKASS',
 'JANOWY MŁYN',
 'KARNKOWO',
 'KB GEOTERMIA UNIEJÓW',
 'KB MANIAC POZNAŃ',
 'KB MARATON TUREK',
 'KLECZEW',
 'KORNELIUSZ SERWIS KOMPUTEROWY',
 'KROJCZYN',
 'KS LZS KRUSZWICA',
 'KS SOKÓŁ GĄSAWA',
 'KS SZOSA ŻNIN',
 'KS ZRYW RADZIEJÓW',
 'KSZYŻTOW',
 'KUJAWSKO-POMORSKIE',
 'KUNA TEAM WŁOCŁAWEK',
 'LECHITA BIEGA KŁECKO',
 'LINGUA PRO BYDGOSZCZ',
 'LIPNO',
 'LKS SPARTA JANOWIEC WLKP',
 'LZS JASTRZĘBIE',
 'MALINOWSCY MIGLASST TEAM'

## Save

In [16]:
def to_seconds(time_str):
    t = datetime.time.fromisoformat(time_str)
    return t.hour * 3600 + t.minute * 60 + t.second

def to_time_str(seconds):
    return datetime.datetime.fromtimestamp(seconds, datetime.UTC).strftime('%T')

def gp_round(n):
    if n - math.floor(n) <= 0.5:
        return math.floor(n)
    return math.ceil(n)

def get_result(row):
    results = [row[col] for col in row.index if col.startswith('result_') and type(row[col]) == str]
    row['total_runs'] = len(results)
    best_6_runs = sorted([to_seconds(r) for r in results])[-6:]
    row['avg_result'] = gp_round(sum(best_6_runs) / len(best_6_runs))
    row['avg_result_str'] = to_time_str(row['avg_result'])
    return row

In [17]:
def get_category(row):
    if pd.isnull(row['year_of_birth']):
        return row

    if row['year_of_birth'] > year - 30:
        row['category'] = row['gender'] + '18'
    elif row['year_of_birth'] > year - 40:
        row['category'] = row['gender'] + '30'
    elif row['year_of_birth'] > year - 50:
        row['category'] = row['gender'] + '40'
    elif row['year_of_birth'] > year - 60:
        row['category'] = row['gender'] + '50'
    elif row['year_of_birth'] > year - 65:
        row['category'] = row['gender'] + '60'
    elif row['year_of_birth'] > year - 70:
        row['category'] = row['gender'] + '65'
    elif row['year_of_birth'] > year - 75:
        row['category'] = row['gender'] + '70'
    elif row['year_of_birth'] > year - 80:
        row['category'] = row['gender'] + '75'
    elif row['year_of_birth'] > year - 86:
        row['category'] = row['gender'] + '80'
    else:
        row['category'] = row['gender'] + '85'

    return row
        
gp = after_3.apply(get_result, axis=1)
gp['year_of_birth'] = gp['year_of_birth'].astype('Int64')
gp = gp.apply(get_category, axis=1)
gp

,name,gender,result_janowiec,result_fabianki,category,result_radziejow,team,year_of_birth,city,total_runs,avg_result,avg_result_str
0,ABRAMOWSKI DAMIAN,M,00:37:45,NaN,M30,NaN,TT SODA,1991,ŻNIN,1,2265,00:37:45
1,ADAMSKA ANNA,K,00:57:09,NaN,K40,NaN,LECHITA BIEGA KŁECKO,1985,KŁECKO,1,3429,00:57:09
2,ADAMSKA IZABELA,K,00:47:33,NaN,K40,NaN,None,1985,JANOWIEC WIELKOPOLSKI,1,2853,00:47:33
3,ADAMSKI ADAM,M,00:47:48,NaN,M30,NaN,None,1986,ŻNIN,1,2868,00:47:48
4,ANDELBRAT BEATA,K,NaN,01:18:33,K50,NaN,GRUDZIADZ,<NA>,NaN,1,4713,01:18:33
...,...,...,...,...,...,...,...,...,...,...,...,...
444,ŚWIĘCICKI ANDRZEJ,M,00:50:43,NaN,M50,NaN,BYDGOSKIE ŻÓŁWIE,1973,BYDGOSZCZ,1,3043,00:50:43
445,ŻABIŃSKI RADOSŁAW,M,NaN,NaN,M30,00:51:29,WKB MARATOŃCZYK WŁOCŁAWEK,1986,WŁOCŁAWEK,1,3089,00:51:29
446,ŻUCHOWSKI BARTOSZ,M,NaN,00:37:34,M18,00:38:14,PŁONNE,1996,PŁONNE,2,2274,00:37:54
447,ŻUCHOWSKI JÓZEF,M,00:41:23,00:40:08,M50,NaN,EKO-OPAŁ PŁONNE,1971,PŁONNE,2,2445,00:40:45


In [18]:
gp.to_sql(name='results', con=cnx, if_exists='replace', index_label='id')

449